In [148]:
# coding=utf-8

import os, sys
import openpyxl

"""
DATA CONFIG
"""
INDEX = 0
NAME = 1
SIZE = 6
AMOUNT = 11
MATERIAL = 12
MESURE = 17
MATERIAL_AMOUNT = 19
GOST = 21
COMMENT = 24

FISRT_LIST = 'Лист1'
FISRT_LIST_FIRST_DATA_ROW = 22
OTHER_LISTS_FIRST_DATA_ROW = 2

"""
SCRIPT CONFIG
"""
DEFAULT_DIR_PATH = os.path.dirname(sys.argv[0])
DEFUALT_RESULT_FILE_NAME = 'output.xls'
DEFAULT_RESULT_FILE_PATH = \
    os.path.join(DEFAULT_DIR_PATH, DEFUALT_RESULT_FILE_NAME)

"""
LOGIC
"""    
def get_files(dir_path=DEFAULT_DIR_PATH):
    """
    Returns list of excel file paths in
    a given directory
    """
    result = []
    for path, subdirs, files in os.walk(dir_path):
        for file in files:
            filename, file_extension = os.path.splitext(file)
            if file_extension in ['.xls', '.xlsx']:
                file_path = os.path.join(path, file)
                print(file_path)
                result.append(file_path)
    return result

def merge(rows):
    """
    Parse a row of the table.
    Add data to result dict.
    
    If this key exists in the dict process its params
    and add the amount
    Else - add new key in dict
    """
    pass

def build_results_file(results_dict, results_file_path=DEFAULT_RESULT_FILE_PATH):
    """
    Build an excel file based on results dict and 
    a given path.
    """
    pass

def process(dir_path=DEFAULT_DIR_PATH, result_file_path=DEFAULT_RESULT_FILE_PATH):
    try:
        files = get_files(dir_path)
        rows_to_process = []
        if files:
            for file in files:
                workbook = openpyxl.load_workbook(filename=file)   
                for sheet in workbook:
                    for row in sheet:
                        row_index = (lambda x: x[0].row)(row)
                        if (sheet is not workbook[FISRT_LIST] \
                        and row_index >= OTHER_LISTS_FIRST_DATA_ROW) \
                            or row_index >= FISRT_LIST_FIRST_DATA_ROW:
                            rows_to_process.append(row) 
                            
            results_dict = merge(rows_to_process)
            # build_results_file(results_dict) 
            # print('Success')
            print(results_dict)
        else:
            print('No files to process')
    
    except Exception as ex:
        print('Error while processing')
        print(ex)


In [149]:
dir_path = '/home/sancau/Desktop/kub'

process(dir_path)


/home/sancau/Desktop/kub/excel/testdata/test.xlsx
None
